In [194]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor,XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score,accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Load datasets
train_df = pd.read_csv('testing/train.csv')
test_df = pd.read_csv('testing/test.csv')

# Combine training and test data to ensure consistent label encoding
combined_df = pd.concat([train_df, test_df], ignore_index=True)

# Separate features and target variable
X_train = train_df.drop(['Severity'], axis=1)
y_train = train_df['Severity']
X_test = test_df.copy()  # Make a copy of test data

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Define preprocessing steps using ColumnTransformer
numeric_transformer = StandardScaler()
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

categorical_cols_train = X_train.select_dtypes(include=['object']).columns.tolist()
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, X_train.select_dtypes(include=['int64', 'float64']).columns),
        ('cat', categorical_transformer, categorical_cols_train)
    ])

# Apply preprocessing to training and test data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)



model = XGBClassifier()
model.fit(X_train_processed, y_train_encoded)
# Make predictions on the preprocessed test data
test_predictions_encoded = model.predict(X_test_processed)

# Inverse transform predicted labels back to original categories
test_predictions = label_encoder.inverse_transform(test_predictions_encoded)

# Create submission dataframe with Ids and predicted Severities
submission_df = pd.DataFrame({
    'Accident_ID': X_test['Accident_ID'],
    'Severity': test_predictions
})

# Save the submission file
submission_df.to_csv('testing/my_submission.csv', index=False)


print(test_predictions)


['Highly_Fatal_And_Damaging' 'Significant_Damage_And_Fatalities'
 'Significant_Damage_And_Serious_Injuries' ...
 'Significant_Damage_And_Serious_Injuries'
 'Significant_Damage_And_Serious_Injuries' 'Highly_Fatal_And_Damaging']
